### Imp Features 0.025 %

In [1]:
import pandas as pd
import numpy as np
import os
import warnings
warnings.filterwarnings('ignore')
from datetime import date
from datetime import datetime
import dateutil
from pandas import *
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import r2_score
from sklearn.model_selection import KFold 
from sklearn.model_selection import cross_val_score
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold

In [3]:
csv_files = os.chdir('C:\\Users\\HP\\Desktop\\PHD Hackathon\\Data Science\\PHD Models\\Weather_Features')
weather_features = pd.read_csv('weatherData_full.csv', encoding='latin 1')
holidays_features = pd.read_csv('holidays_features.csv')
economic_features = pd.read_csv('economic_features.csv')
womenClothing_sales = pd.read_csv('womenClothing_sales.csv')

In [4]:
print('womenClothing :', womenClothing_sales.shape)
print('holidays_features :', holidays_features.shape)
print('economic_features :', economic_features.shape)
print('weather_features :', weather_features.shape)

womenClothing : (84, 3)
holidays_features : (81, 6)
economic_features : (96, 18)
weather_features : (96, 83)


## Seperate Train and Test Data

### Test Data

In [5]:
weather_features_test = weather_features[weather_features['Year'] == 2016]
holidays_features_test = holidays_features[holidays_features['Year'] == 2016]
economic_features_test = economic_features[economic_features['Year'] == 2016]

In [6]:
print('holidays_features :', holidays_features_test.shape)
print('economic_features :', economic_features_test.shape)
print('weather_features :', weather_features_test.shape)

holidays_features : (11, 6)
economic_features : (12, 18)
weather_features : (12, 83)


In [7]:
test_data = weather_features_test.merge(holidays_features_test, left_on=['Year', 'Month'], right_on=['Year', 'Month'], how='left')
test_data = test_data.merge(economic_features_test, left_on=['Year', 'Month'], right_on=['Year', 'Month'], how='left')

In [8]:
test_data.tail()

,Year,Month,Temp high (°C)min,Temp high (°C)max,Temp high (°C)mean,Temp high (°C)var,Temp avg (°C)min,Temp avg (°C)max,Temp avg (°C)mean,Temp avg (°C)var,...,Earnings or wages in dollars per hour,AdvertisingExpenses (in Thousand Dollars),Cotton Monthly Price - US cents per Pound(lbs),Change(in%),Average upland planted(million acres),Average upland harvested(million acres),yieldperharvested acre,Production (in 480-lb netweright in million bales),Mill use (in 480-lb netweright in million bales),Exports
7,2016,5,11.0,33.0,21.258065,45.197849,9.0,28.0,17.193548,31.894624,...,25.68,0.0,70.28,0.73,8.422,7.920,755,12.455,3.275,8.50
8,2016,6,19.0,31.0,27.100000,11.127586,16.0,27.0,22.433333,6.322989,...,25.42,0.0,74.10,0.73,9.347,8.588,792,12.165,3.275,9.95
9,2016,7,22.0,36.0,30.129032,11.449462,21.0,31.0,26.064516,6.662366,...,25.53,0.0,79.50,0.73,9.824,9.104,802,15.215,3.275,10.95
10,2016,8,26.0,36.0,30.354839,6.169892,22.0,32.0,26.354839,5.303226,...,25.52,0.0,79.50,0.00,9.824,9.340,787,15.314,3.275,10.95
11,2016,9,15.0,33.0,26.166667,19.040230,14.0,28.0,22.133333,12.809195,...,25.74,0.0,77.86,0.00,9.950,9.464,790,15.580,3.275,10.95


In [9]:
test_data.isnull().sum()

Year                                                                 0
Month                                                                0
Temp high (°C)min                                                    0
Temp high (°C)max                                                    0
Temp high (°C)mean                                                   0
Temp high (°C)var                                                    0
Temp avg (°C)min                                                     0
Temp avg (°C)max                                                     0
Temp avg (°C)mean                                                    0
Temp avg (°C)var                                                     0
Temp low (°C)min                                                     0
Temp low (°C)max                                                     0
Temp low (°C)mean                                                    0
Temp low (°C)var                                                     0
Dew Po

### Train Data

In [10]:
weather_features_train = weather_features[weather_features['Year'] != 2016]
holidays_features_train = holidays_features[holidays_features['Year'] != 2016]
economic_features_train = economic_features[economic_features['Year'] != 2016]

In [11]:
print('holidays_features :', holidays_features_train.shape)
print('economic_features :', economic_features_train.shape)
print('weather_features :', weather_features_train.shape)

holidays_features : (70, 6)
economic_features : (84, 18)
weather_features : (84, 83)


In [12]:
train_data = womenClothing_sales.merge(holidays_features_train, left_on=['Year', 'Month'], right_on=['Year', 'Month'], how='left')
train_data = train_data.merge(economic_features_train, left_on=['Year', 'Month'], right_on=['Year', 'Month'], how='left')
train_data = train_data.merge(weather_features_train, left_on=['Year', 'Month'], right_on=['Year', 'Month'], how='left')

In [13]:
train_data.head()

,Year,Month,WomenClothing_Sales,Event,Federal Holiday,Event_Yes_No,Federal Holiday_Yes_No,Monthly Nominal GDP Index (inMillion$),Monthly Real GDP Index (inMillion$),CPI,...,Wind (km/h) highvar,Fog,"Fog , Rain","Fog , Rain , Snow","Fog , Snow",Other,Rain,"Rain , Snow",Snow,Thunderstorm
0,2009,1,1755.0,0.0,2.0,No,Yes,14421.752895,14407.053343,233.402,...,96.625806,0,0,2,2,21,3,0,3,0
1,2009,2,1729.0,1.0,1.0,Yes,Yes,14389.200466,14366.176571,234.663,...,167.349206,0,0,0,1,22,4,1,0,0
2,2009,3,2256.0,NaN,NaN,NaN,NaN,14340.701639,14351.786822,235.067,...,92.245161,1,0,0,2,21,7,0,0,0
3,2009,4,2662.0,1.0,0.0,Yes,No,14326.815525,14351.601731,235.582,...,87.426437,0,1,0,0,18,11,0,0,0
4,2009,5,2732.0,1.0,1.0,Yes,Yes,14345.904809,14368.123959,235.975,...,88.874886,0,4,0,0,17,10,0,0,0


In [14]:
train_data.shape

(84, 104)

In [15]:
train_data.isnull().sum()

Year                                                                  0
Month                                                                 0
WomenClothing_Sales                                                   0
Event                                                                14
Federal Holiday                                                      14
Event_Yes_No                                                         14
Federal Holiday_Yes_No                                               14
Monthly Nominal GDP Index (inMillion$)                                0
Monthly Real GDP Index (inMillion$)                                   0
CPI                                                                   0
unemployment rate                                                     0
CommercialBankInterestRateonCreditCardPlans                           0
Finance Rate on Personal Loans at Commercial Banks, 24 Month Loan     0
Earnings or wages  in dollars per hour                          

In [16]:
# train_data['new_col'] = train_data['Month'].apply(lambda x: 1 if x==12 else 0)
# test_data['new_col'] = test_data['Month'].apply(lambda x: 1 if x==12 else 0)


### Drop Columns

In [16]:
X = train_data.copy().drop(['WomenClothing_Sales','Event_Yes_No', 'Federal Holiday_Yes_No'], axis = 1)
Y = train_data['WomenClothing_Sales']
Y1 = train_data['Year']

### Fill Missing Values

In [17]:
X = X.fillna(0)
test_data = test_data.fillna(0)

## Models

### Train Val Split

In [18]:
trainX_fold, valX_fold, trainY_fold, valY_fold =  train_test_split(X, Y1, test_size=0.25, random_state=42, shuffle=False)
y_train, y_val = Y.loc[trainY_fold.index], Y.loc[valY_fold.index]

In [19]:
print(trainX_fold.shape)
type(y_train)

(63, 101)


pandas.core.series.Series

### MAPE Function

In [20]:
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

### Light GBM

In [21]:
def run_lgb(train_X, train_y, val_X, val_y):
    param = {'num_leaves': 30,
#              'min_data_in_leaf': 100,
             'objective': 'regression',
             'max_depth': 8,
             'learning_rate': 0.01,
             "boosting": "gbdt",
             "feature_fraction": 0.7,
             "bagging_freq": 1,
             "bagging_fraction": 0.7,
             "bagging_seed": 11,
             "metric": 'mape',
             # "lambda_l1": 0.2634,
             "random_state": 133,
             "verbosity": -1}

    lgtrain = lgb.Dataset(train_X, label=train_y)
    lgval = lgb.Dataset(val_X, label=val_y)
    evals_result = {}
    model = lgb.train(param, lgtrain, 2000, valid_sets=[lgval], early_stopping_rounds=100, verbose_eval=100,
                      evals_result=evals_result)
    return model, evals_result


In [22]:
model, evals_result = run_lgb(trainX_fold, y_train, valX_fold, y_val)

Training until validation scores don't improve for 100 rounds.
[100]	valid_0's mape: 0.195624
[200]	valid_0's mape: 0.181742
[300]	valid_0's mape: 0.175896
[400]	valid_0's mape: 0.170129
[500]	valid_0's mape: 0.167004
[600]	valid_0's mape: 0.162919
[700]	valid_0's mape: 0.161281
[800]	valid_0's mape: 0.159466
[900]	valid_0's mape: 0.157624
[1000]	valid_0's mape: 0.158465
Early stopping, best iteration is:
[968]	valid_0's mape: 0.157335


### Predict On Test

In [23]:
test_data = test_data.drop(['Event_Yes_No', 'Federal Holiday_Yes_No'], axis=1)

In [24]:
model.predict(test_data)

array([2402.33592895, 2992.96830379, 2998.30310342, 2762.06962791,
       2402.33592895, 2632.1218568 , 2595.92314856, 2779.97998235,
       3019.31766963, 3161.78041328, 3232.59558709, 3121.02598789])

In [25]:
test_results = pd.DataFrame({'Sales(In ThousandDollars)': model.predict(test_data)})

In [26]:
test_results

,Sales(In ThousandDollars)
0,2402.335929
1,2992.968304
2,2998.303103
3,2762.069628
4,2402.335929
5,2632.121857
6,2595.923149
7,2779.979982
8,3019.317670
9,3161.780413


In [27]:
submission = pd.read_csv('C:\\Users\\HP\\Desktop\\PHD Hackathon\\Data Science\\NewSampleSubmission.csv')
submission.head()

,ID,Sales(In ThousandDollars),Year,Month,ProductCategory
0,1,110,2016,1,WomenClothing
1,2,140,2016,2,WomenClothing
2,3,199,2016,3,WomenClothing
3,4,246,2016,4,WomenClothing
4,5,66,2016,5,WomenClothing


In [28]:
submission['Sales(In ThousandDollars)'] = test_results['Sales(In ThousandDollars)']

In [29]:
submission

,ID,Sales(In ThousandDollars),Year,Month,ProductCategory
0,1,2402.335929,2016,1,WomenClothing
1,2,2992.968304,2016,2,WomenClothing
2,3,2998.303103,2016,3,WomenClothing
3,4,2762.069628,2016,4,WomenClothing
4,5,2402.335929,2016,5,WomenClothing
5,6,2632.121857,2016,6,WomenClothing
6,7,2595.923149,2016,7,WomenClothing
7,8,2779.979982,2016,8,WomenClothing
8,9,3019.317670,2016,9,WomenClothing
9,10,3161.780413,2016,10,WomenClothing


In [30]:
#submission.to_csv('C:\\Users\\HP\\Desktop\\trial\\regression_lgbm_submission.csv', index=False)

In [31]:
############################# ---------------------------------------------------------- #################################

## Random Forest

In [32]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor

rf_model = RandomForestRegressor(bootstrap=True, criterion='mae', max_depth=3,
                                   max_features='sqrt', max_leaf_nodes=None,
                                   min_samples_leaf=2, min_samples_split=3, n_estimators=20, n_jobs=1,)

rf_model.fit(trainX_fold , y_train)

y_pred = rf_model.predict(valX_fold)
mean_absolute_percentage_error(y_val, y_pred)

17.870623603636442

### Take Imp_Features

In [33]:
X_cols = trainX_fold.columns
feature_imp_RFC = pd.DataFrame({"imp_features":rf_model.feature_importances_, "columns":X_cols})
feature_imp_RFC.sort_values(by = 'imp_features', ascending=False)

,columns,imp_features
4,Monthly Nominal GDP Index (inMillion$),0.088941
6,CPI,0.078321
5,Monthly Real GDP Index (inMillion$),0.077677
1,Month,0.075098
28,Temp low (°C)min,0.062664
36,Dew Point avg (°C)min,0.055171
9,Finance Rate on Personal Loans at Commercial B...,0.039411
90,Wind (km/h) highmean,0.029035
2,Event,0.024169
29,Temp low (°C)max,0.023352


In [34]:
top_features = feature_imp_RFC[feature_imp_RFC['imp_features'] >= 0.025]
top_features.head()

,columns,imp_features
1,Month,0.075098
4,Monthly Nominal GDP Index (inMillion$),0.088941
5,Monthly Real GDP Index (inMillion$),0.077677
6,CPI,0.078321
9,Finance Rate on Personal Loans at Commercial B...,0.039411


In [35]:
top_features.shape

(8, 2)

#### Selecting Imp Features From RF

In [36]:
imp_columns_train = pd.DataFrame()
for i in top_features['columns'].values:
    imp_columns_train[i] = trainX_fold[i]
    
imp_columns_train.head()

,Month,Monthly Nominal GDP Index (inMillion$),Monthly Real GDP Index (inMillion$),CPI,"Finance Rate on Personal Loans at Commercial Banks, 24 Month Loan",Temp low (°C)min,Dew Point avg (°C)min,Wind (km/h) highmean
0,1,14421.752895,14407.053343,233.402,11.44,-14.0,-20.0,39.322581
1,2,14389.200466,14366.176571,234.663,11.05,-11.0,-18.0,41.642857
2,3,14340.701639,14351.786822,235.067,11.05,-11.0,-18.0,38.612903
3,4,14326.815525,14351.601731,235.582,11.05,1.0,-9.0,39.566667
4,5,14345.904809,14368.123959,235.975,11.25,6.0,0.0,32.838980


In [37]:
imp_columns_val = pd.DataFrame()

for i in top_features['columns'].values:
    imp_columns_val[i] = valX_fold[i]

imp_columns_val.head()

,Month,Monthly Nominal GDP Index (inMillion$),Monthly Real GDP Index (inMillion$),CPI,"Finance Rate on Personal Loans at Commercial Banks, 24 Month Loan",Temp low (°C)min,Dew Point avg (°C)min,Wind (km/h) highmean
63,4,17208.864684,15870.178053,259.985,10.09,-1.0,-9.0,39.166667
64,5,17345.334028,15946.741336,261.225,9.57,9.0,-2.0,35.451748
65,6,17408.564289,15990.977418,261.350,9.57,13.0,4.0,29.900000
66,7,17551.781065,16089.499187,261.498,9.57,17.0,9.0,37.161561
67,8,17684.635349,16196.420132,261.075,10.73,16.0,9.0,30.064516


### train Model with Imp_Features

In [38]:
rf_model.fit(imp_columns_train, y_train)
y_pred1 = rf_model.predict(imp_columns_val)

mean_absolute_percentage_error(y_val, y_pred1)

15.344611429914284

### Predict Test Data

In [39]:
imp_columns_test = pd.DataFrame()

for i in top_features['columns'].values:
    imp_columns_test[i] = test_data[i]

imp_columns_test.head()

,Month,Monthly Nominal GDP Index (inMillion$),Monthly Real GDP Index (inMillion$),CPI,"Finance Rate on Personal Loans at Commercial Banks, 24 Month Loan",Temp low (°C)min,Dew Point avg (°C)min,Wind (km/h) highmean
0,1,18286.638523,16546.153123,260.342,9.66,-12.0,-19.0,42.354974
1,10,18740.780023,16725.460098,264.738,9.64,3.0,-4.0,34.612903
2,11,18960.461568,16910.896330,265.203,9.45,1.0,-8.0,36.800000
3,12,19015.393408,16918.050624,265.421,9.45,-8.0,-18.0,39.871103
4,2,18241.713812,16503.500358,260.875,10.03,-18.0,-27.0,44.103448


In [40]:
rf_model.predict(imp_columns_test)

array([3022.4      , 3543.0578125, 3566.7171875, 3522.715625 ,
       3141.05     , 3697.3171875, 3669.1671875, 3543.0578125,
       3543.0578125, 3470.9515625, 3470.9515625, 3543.0578125])

In [41]:
test_predictions = pd.DataFrame({'Sales(In ThousandDollars)': rf_model.predict(imp_columns_test)})
test_predictions.head(2)

,Sales(In ThousandDollars)
0,3022.400000
1,3543.057813


In [42]:
submissionfile = pd.read_csv('C:\\Users\\HP\\Desktop\\PHD Hackathon\\Data Science\\NewSampleSubmission.csv')
submissionfile.head(2)

,ID,Sales(In ThousandDollars),Year,Month,ProductCategory
0,1,110,2016,1,WomenClothing
1,2,140,2016,2,WomenClothing


In [43]:
submissionfile['Sales(In ThousandDollars)'] = test_predictions['Sales(In ThousandDollars)']
submissionfile

,ID,Sales(In ThousandDollars),Year,Month,ProductCategory
0,1,3022.400000,2016,1,WomenClothing
1,2,3543.057813,2016,2,WomenClothing
2,3,3566.717187,2016,3,WomenClothing
3,4,3522.715625,2016,4,WomenClothing
4,5,3141.050000,2016,5,WomenClothing
5,6,3697.317187,2016,6,WomenClothing
6,7,3669.167188,2016,7,WomenClothing
7,8,3543.057813,2016,8,WomenClothing
8,9,3543.057813,2016,9,WomenClothing
9,10,3470.951562,2016,10,WomenClothing


In [44]:
#submissionfile.to_csv('C:\\Users\\HP\\Desktop\\trial\\regression_rf_submission.csv', index=False)

### RF Tuning

In [45]:
param_grid = {
    'bootstrap': [True],
    'max_depth': [3, 5, 8, 10, 12],
    'max_features': ['sqrt', 'log2', 'auto'],
    'min_samples_leaf': [2, 3, 4, 5],
    'min_samples_split': [3, 5, 8, 10, 12],
    'n_estimators': [20, 30, 40, 50]
}

rf = RandomForestRegressor()
rf_grid = GridSearchCV(rf, param_grid = param_grid, cv = 5, n_jobs = -1)
rf_grid.fit(trainX_fold, y_train)

C:\Users\HP\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'bootstrap': [True], 'max_depth': [3, 5, 8, 10, 12], 'max_features': ['sqrt', 'log2', 'auto'], 'min_samples_leaf': [2, 3, 4, 5], 'min_samples_split': [3, 5, 8, 10, 12], 'n_estimators': [20, 30, 40, 50]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [50]:
rf_grid.best_params_

{'bootstrap': True,
 'max_depth': 10,
 'max_features': 'auto',
 'min_samples_leaf': 2,
 'min_samples_split': 12,
 'n_estimators': 30}

### Build RF Model With Best Parameters

In [51]:
rf_model_best = RandomForestRegressor(bootstrap=True, max_depth=10, max_features='auto', min_samples_leaf=2, 
                                     min_samples_split=12, n_estimators=30)

rf_model_best.fit(imp_columns_train, y_train)
preds = rf_model_best.predict(imp_columns_val)
mean_absolute_percentage_error(y_val, preds)

13.191695513521118

In [52]:
test_preds_rf = pd.DataFrame({'Sales(In ThousandDollars)': rf_model_best.predict(imp_columns_test)})
test_preds_rf

,Sales(In ThousandDollars)
0,2960.227548
1,3694.830723
2,3752.782511
3,4415.136999
4,3135.757695
5,3732.792751
6,3719.320554
7,3698.627920
8,3673.581987
9,3611.332369


In [53]:
submissionfile['Sales(In ThousandDollars)'] = test_preds_rf['Sales(In ThousandDollars)']
submissionfile

,ID,Sales(In ThousandDollars),Year,Month,ProductCategory
0,1,2960.227548,2016,1,WomenClothing
1,2,3694.830723,2016,2,WomenClothing
2,3,3752.782511,2016,3,WomenClothing
3,4,4415.136999,2016,4,WomenClothing
4,5,3135.757695,2016,5,WomenClothing
5,6,3732.792751,2016,6,WomenClothing
6,7,3719.320554,2016,7,WomenClothing
7,8,3698.627920,2016,8,WomenClothing
8,9,3673.581987,2016,9,WomenClothing
9,10,3611.332369,2016,10,WomenClothing


In [54]:
#submissionfile.to_csv('C:\\Users\\HP\\Desktop\\trial\\regression_rf_best_submission.csv', index=False)

In [55]:
##################################################################################################################

### GBM

In [56]:
from sklearn.ensemble import GradientBoostingRegressor

model_gbr = GradientBoostingRegressor()
model_gbr.fit(trainX_fold, y_train)

val_predictions = model_gbr.predict(valX_fold)
mean_absolute_percentage_error(y_val, val_predictions)

14.523764955222157

In [57]:
test_preds = pd.DataFrame({'Sales(In ThousandDollars)': model_gbr.predict(test_data)})
test_preds

,Sales(In ThousandDollars)
0,2282.923243
1,2794.615967
2,2745.423699
3,2531.570767
4,2282.923243
5,2662.442831
6,2638.550659
7,2737.947766
8,2770.363805
9,2775.460469


### GBM With rf_model Imp_features

In [58]:
from sklearn.ensemble import GradientBoostingRegressor

model_gbr = GradientBoostingRegressor()
model_gbr.fit(imp_columns_train, y_train)

val_predictions = model_gbr.predict(imp_columns_val)
mean_absolute_percentage_error(y_val, val_predictions)

7.408293763944457

In [59]:
test_preds1 = pd.DataFrame({'Sales(In ThousandDollars)': model_gbr.predict(imp_columns_test)})

In [60]:
submissionfile['Sales(In ThousandDollars)'] = test_preds1['Sales(In ThousandDollars)']
submissionfile

,ID,Sales(In ThousandDollars),Year,Month,ProductCategory
0,1,3175.715924,2016,1,WomenClothing
1,2,4081.715112,2016,2,WomenClothing
2,3,4147.339618,2016,3,WomenClothing
3,4,4973.584250,2016,4,WomenClothing
4,5,3659.233177,2016,5,WomenClothing
5,6,4105.666727,2016,6,WomenClothing
6,7,4191.583765,2016,7,WomenClothing
7,8,4331.774975,2016,8,WomenClothing
8,9,3988.001133,2016,9,WomenClothing
9,10,3714.814067,2016,10,WomenClothing


In [70]:
#submissionfile.to_csv('C:\\Users\\HP\\Desktop\\trial\\regression_gbm_submission.csv', index=False)

### GBM With GridSearch

In [63]:
parameters = {
    "criterion": ["friedman_mse",'mae'],
    "loss":['ls', 'lad', 'huber', 'quantile'],
    "learning_rate": [0.025, 0.05, 0.075, 0.1, 0.15, 0.2, 0.25],
    "max_depth":[3,5,8],
    "n_estimators":[10, 15, 20],
    "min_samples_split":[2,3,4,5,6]
    }

gbc = GradientBoostingRegressor()
gbc_grid = GridSearchCV(gbc, parameters, cv=5, n_jobs=-1)
gbc_grid.fit(imp_columns_train, y_train)

C:\Users\HP\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=3, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_sampl...=None, subsample=1.0, tol=0.0001,
             validation_fraction=0.1, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'criterion': ['friedman_mse', 'mae'], 'loss': ['ls', 'lad', 'huber', 'quantile'], 'learning_rate': [0.025, 0.05, 0.075, 0.1, 0.15, 0.2, 0.25], 'max_depth': [3, 5, 8], 'n_estimators': [10, 15, 20], 'min_samples_split': [2, 3, 4, 5, 6]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [64]:
gbc_grid.best_params_

{'criterion': 'mae',
 'learning_rate': 0.25,
 'loss': 'lad',
 'max_depth': 8,
 'min_samples_split': 4,
 'n_estimators': 20}

In [65]:
gbc_best = GradientBoostingRegressor(criterion='mae',
                                        learning_rate = 0.25,
                                        loss = 'lad',
                                        max_depth = 8,
                                        min_samples_split = 4,
                                        n_estimators =20)
gbc_best.fit(imp_columns_train, y_train)

GradientBoostingRegressor(alpha=0.9, criterion='mae', init=None,
             learning_rate=0.25, loss='lad', max_depth=8,
             max_features=None, max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=1, min_samples_split=4,
             min_weight_fraction_leaf=0.0, n_estimators=20,
             n_iter_no_change=None, presort='auto', random_state=None,
             subsample=1.0, tol=0.0001, validation_fraction=0.1, verbose=0,
             warm_start=False)

In [66]:
y_cap = gbc_best.predict(imp_columns_val)
mean_absolute_percentage_error(y_val, y_cap)

15.196573939929431

In [67]:
gbc_best.predict(imp_columns_test)

array([2844.02099444, 3605.15629086, 3666.59077435, 4153.46728646,
       3398.50632517, 3667.75685448, 3680.80096777, 3697.79326377,
       3632.4878217 , 3326.24075456, 3292.17921205, 3489.46994444])

In [68]:
test_preds2 = pd.DataFrame({'Sales(In ThousandDollars)': gbc_best.predict(imp_columns_test)})
test_preds2

,Sales(In ThousandDollars)
0,2844.020994
1,3605.156291
2,3666.590774
3,4153.467286
4,3398.506325
5,3667.756854
6,3680.800968
7,3697.793264
8,3632.487822
9,3326.240755


In [69]:
submissionfile['Sales(In ThousandDollars)'] = test_preds2['Sales(In ThousandDollars)']
submissionfile

,ID,Sales(In ThousandDollars),Year,Month,ProductCategory
0,1,2844.020994,2016,1,WomenClothing
1,2,3605.156291,2016,2,WomenClothing
2,3,3666.590774,2016,3,WomenClothing
3,4,4153.467286,2016,4,WomenClothing
4,5,3398.506325,2016,5,WomenClothing
5,6,3667.756854,2016,6,WomenClothing
6,7,3680.800968,2016,7,WomenClothing
7,8,3697.793264,2016,8,WomenClothing
8,9,3632.487822,2016,9,WomenClothing
9,10,3326.240755,2016,10,WomenClothing


In [85]:
#submissionfile.to_csv('C:\\Users\\HP\Desktop\\trial\\regression_gbm_best_submission.csv', index=False)

### Linear Regression

In [70]:
from sklearn import linear_model
reg_lasso = linear_model.Lasso(alpha=0.1)

reg_lasso.fit(trainX_fold, y_train)
Y_cap  = reg_lasso.predict(valX_fold)

mean_absolute_percentage_error(y_val, Y_cap)

C:\Users\HP\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


27.418520794265227

In [71]:
test_preds_reg = pd.DataFrame({'Sales(In ThousandDollars)':reg_lasso.predict(test_data)})
test_preds_reg

,Sales(In ThousandDollars)
0,-3.047104e+06
1,-3.055305e+06
2,-3.092860e+06
3,-3.096222e+06
4,-3.017411e+06
5,-3.055895e+06
6,-3.052783e+06
7,-3.029324e+06
8,-3.045857e+06
9,-3.046832e+06


### Linear Regression With Imp Features

In [72]:
reg_lasso_imp = linear_model.Lasso(alpha=0.1)

reg_lasso_imp.fit(imp_columns_train, y_train)
Y_cap1  = reg_lasso_imp.predict(imp_columns_val)

mean_absolute_percentage_error(y_val, Y_cap1)

C:\Users\HP\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


9.803498612793032

In [73]:
test_preds_reg_imp = pd.DataFrame({'Sales(In ThousandDollars)':reg_lasso_imp.predict(imp_columns_test)})
test_preds_reg_imp

,Sales(In ThousandDollars)
0,3363.839067
1,4210.020919
2,4648.079244
3,4922.015725
4,3733.009906
5,4071.184366
6,3944.847829
7,3721.079991
8,3800.161825
9,3667.219277


In [74]:
lr_model = linear_model.Lasso(alpha=[0.1])
parameters = {'fit_intercept':[True,False], 'normalize':[True,False], 'copy_X':[True, False]}

reg_grid = GridSearchCV(lr_model, parameters, cv=5)
reg_grid.fit(imp_columns_train, y_train)


C:\Users\HP\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
C:\Users\HP\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
C:\Users\HP\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
C:\Users\HP\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the numbe

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=Lasso(alpha=[0.1], copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'fit_intercept': [True, False], 'normalize': [True, False], 'copy_X': [True, False]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [75]:
reg_grid.best_params_

{'copy_X': True, 'fit_intercept': True, 'normalize': False}

In [76]:
reg_lasso_best = linear_model.Lasso(alpha=0.1, copy_X=True, fit_intercept=True, normalize=True)

reg_lasso_best.fit(imp_columns_train, y_train)
Y_cap2  = reg_lasso_best.predict(imp_columns_val)

mean_absolute_percentage_error(y_val, Y_cap2)

C:\Users\HP\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


8.776754985449056

In [77]:
test_preds_reg_best = pd.DataFrame({'Sales(In ThousandDollars)':reg_lasso_best.predict(imp_columns_test)})
test_preds_reg_best

,Sales(In ThousandDollars)
0,3493.586440
1,4379.352752
2,4789.667078
3,5068.063593
4,3850.549998
5,4195.425775
6,4068.051288
7,3858.946168
8,3953.811525
9,3843.442062


In [ ]:
######################################################

In [78]:
imp_columns_train.columns

Index(['Month', 'Monthly Nominal GDP Index (inMillion$)',
       'Monthly Real GDP Index (inMillion$)', 'CPI',
       'Finance Rate on Personal Loans at Commercial Banks, 24 Month Loan',
       'Temp low (°C)min', 'Dew Point avg (°C)min', 'Wind (km/h) highmean'],
      dtype='object')

In [ ]:
women_df[['temp_avg','dew_point_avg','pressure_avg','mon_nom_GDP','mon_real_GDP','CPI','unemp_rate','cc_intrest','per_loan_intrest_rate_24 ',
 'wage_per_hour','Cotton_Monthly_Price','Change(in%)','Exports','Event']]

'Monthly Nominal GDP Index (inMillion$)', 
'Monthly Real GDP Index (inMillion$)'

In [93]:
other_imp = ['Monthly Nominal GDP Index (inMillion$)', 'Monthly Real GDP Index (inMillion$)', 'CPI',
                      'Event', 'Change(in%)', 'Cotton Monthly Price - US cents per Pound(lbs)', 'Temp avg (°C)min',
                      'Dew Point avg (°C)min', 'CommercialBankInterestRateonCreditCardPlans' ,
                      'Finance Rate on Personal Loans at Commercial Banks, 24 Month Loan', 'Earnings or wages in dollars per hour',
                      'Exports', 'Sea Level Press avgmean']

In [ ]:
new_df = pd.DataFrame()

new_df